In [47]:
import pandas as pd
import json

In [48]:
def recall(gold, pred):
    return len(set(gold) & set(pred)) / len(gold)

def precision(gold, pred):
    return len(set(gold) & set(pred)) / len(pred)
    
def f1(gold, pred):
    r = recall(gold, pred)
    p = precision(gold, pred)
    return 2 * r * p / (r + p)

In [54]:
test_file = "IQG_test.csv"
test_df = pd.read_csv(test_file, sep=',', header=0)
test_df = test_df[test_df['label'] == 'O'].reset_index(drop=True)
print(test_df.shape)

generation_result_file = 'test_8.csv'
generation_reuslt_df = pd.read_csv(generation_result_file, sep=',', header=0)
print(generation_reuslt_df.shape)

new_test_df = pd.concat([test_df, generation_reuslt_df], axis=1)
print(new_test_df.shape)

check_df = new_test_df[new_test_df['gold'] == new_test_df['additional_question']]
print(check_df.shape)


scene_graphs_file = "./sceneGraphs/train_sceneGraphs.json"

with open(scene_graphs_file, 'r') as f:
    scene_graphs = json.load(f)


for idx, example in new_test_df.iterrows():
    scene_graph = scene_graphs[str(example['image_id'])]
    image_entitis = []
    
    
    for obj_key, object_value in scene_graph['objects'].items():
        image_entitis.append(object_value['name'])
        image_entitis = list(set(image_entitis))
    
    gold_entities = []
    generation_entities = []
    for entity in image_entitis:
        if entity in example['additional_question']:
            gold_entities.append(entity)
            
        if entity in example['generation']:
            generation_entities.append(entity)
            
    
    precision_score = precision(gold_entities, generation_entities)
    recall_score = recall(gold_entities, generation_entities)
    f1_score = f1(gold_entities, generation_entities)
    
    new_test_df.loc[idx, 'precision'] = precision_score
    new_test_df.loc[idx, 'recall'] = recall_score
    new_test_df.loc[idx, 'f1'] = f1_score

unique_additional_questions = new_test_df['generation'].unique()
print(unique_additional_questions.shape)

argmax_f1_list = []
recall_list = []
precision_list = []

for unique_additional_question in unique_additional_questions:
    same_additional_question_df = new_test_df[new_test_df['generation'] == unique_additional_question]
    
    get_argmax_idx = same_additional_question_df['f1'].argmax()
    
    argmax_f1_list.append(same_additional_question_df.iloc[get_argmax_idx]['f1'])
    recall_list.append(same_additional_question_df.iloc[get_argmax_idx]['recall'])
    precision_list.append(same_additional_question_df.iloc[get_argmax_idx]['precision'])
    
average_f1 = sum(argmax_f1_list) / len(argmax_f1_list)
average_recall = sum(recall_list) / len(recall_list)
average_precision = sum(precision_list) / len(precision_list)

print(average_precision)
print(average_recall)
print(average_f1)

    

(787, 11)
(787, 2)
(787, 13)
(787, 13)
(351,)
0.6818613485280152
0.7996201329534662
0.7008682675349326
